<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_25_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Посчитать количество установок по месяцам.

2. Найти пользователей, установивших приложение в марте, и сделавших покупку в течение 30 дней.

3. Вычислить разницу между первой и последней покупкой по каждому пользователю.

4. Построить когортную таблицу: месяц установки × количество покупателей через 0, 1, 2 месяца.
